# Blabber Cleaning
#### (By: Mark Ehab Aziz)
#### (Built Under: Python 3.11.4)
Filtering out and cleaning text data.
As tasked inside the 'to do.txt'

In [65]:
# Importing Libraries
import pandas as pd     # Loading Data
import re
from nltk.tokenize import regexp_tokenize
from nltk.tokenize import word_tokenize     # To Tokenize Words
from nltk.stem import PorterStemmer         # Stemming words
from nltk.stem import WordNetLemmatizer     # Lemmatizing words

In [66]:
# Loading data into kernel
# Using two methods (As stated in my previous projects)
# 1. Path working within my git repo
blab = pd.read_csv("../dataset/train.csv")

# 2. Path when data is within the same folder
#blab = pd.read_csv("./train.csv")

# Data Exploration
Using `.head(n)` to show the first $n^{th}$ rows of the dataset.

In [67]:
# Defining n rows to see
n = 5

# Showing head
blab.head(n)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\r\nWhy the edits made under my use...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\r\nMore\r\nI can't make any real suggestions...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


As stated by our todo list, we are only tasked with cleaning of the text, so we'll be focusing on `comment_text`.

Referring to our todo list once again, we will be dropping `id`, `toxic`, `severe_toxic`, `obscene`, `threat`, `insult`, and `identity_hate`; as we are not concerned with classifying the sentiment or the meaning behind any of the comments.

Reminder for what to be done:
- Read Text
- Clean Text (Capitalisation, punctuation)
- Remove Stop Words
- Tokenization
- Stemming

Under no aforementioned task will we be using the columns I have mentioned to drop.

In [68]:
# Defining list of columns to be dropped
col_droppable = ["id", "toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

# Dropping
txt_blab = blab.drop(columns = col_droppable)

# Viewing
txt_blab.head(n)

,comment_text
0,Explanation\r\nWhy the edits made under my use...
1,D'aww! He matches this background colour I'm s...
2,"Hey man, I'm really not trying to edit war. It..."
3,"""\r\nMore\r\nI can't make any real suggestions..."
4,"You, sir, are my hero. Any chance you remember..."


In [69]:
# Removing '\n' '\r' '\t' from every line-kinda
txt_blab.replace(r'[\r\n\t]', ' ', regex = True, inplace=True)

# Cleaning Above Sentences
Using NLTK and Regex.

In [98]:
# Defining Regex patterns
# Match words starting with Uppercase letters
upper_words = r"([A-Z])\w+"

# Match Words that start with either Upper/lowercase letters
upper_lower_words = r"[A-Za-z]\w+"

# Match Emails
email_pattern = r"\b[\w\.-]+@[\w\.-]+\.\w{2,4}\b"

# Match URLs
url_pattern = r"(http|ftp|https):\/\/([\w+?\.\w+])+([a-zA-Z0-9\~\!\@\#\$\%\^\&\*\(\)_\-\=\+\\\/\?\.\:\;\'\,]*)?"

In [104]:
# Instantiate a bag of words list, to hold tokens of each word
# Probably better to use a dictionary if we care about count???
# Still gotta access every token and change to lower
# Also still gotta make it so that i can stem and remove some other special characters
# Progress?????
bag_o_words = []

# Start and finish indecies of iterator
# Bound to become the length of the file eventually
# I really dispise this
for i in range(start, fin):
    line = txt_blab.iloc[i,0]
    print(line)

    bag_o_words.append(regexp_tokenize(line, upper_lower_words))

Ahh, Hello Witzeman     203.92.84.161    Symbols: ~ | ¡ ¿ † ‡ ↔ ↑ ↓ • ¶   # ½ ⅓ ⅔ ¼ ¾ ⅛ ⅜ ⅝ ⅞ ∞   ‘ “ ’ ” «»   ¤ ₳ ฿ ₵ ¢ ₡ ₢ $ ₫ ₯ € ₠ ₣ ƒ ₴ ₭ ₤ ℳ ₥ ₦ № ₧ ₰ £ ៛ ₨ ₪ ৳ ₮ ₩ ¥   ♠ ♣ ♥ ♦   m² m³   Characters: Á á Ć ć É é Í í Ĺ ĺ Ń ń Ó ó Ŕ ŕ Ś ś Ú ú Ý ý Ź ź   À à È è Ì ì Ò ò Ù ù   Â â Ĉ ĉ Ê ê Ĝ ĝ Ĥ ĥ Î î Ĵ ĵ Ô ô Ŝ ŝ Û û Ŵ ŵ Ŷ ŷ   Ä ä Ë ë Ï ï Ö ö Ü ü Ÿ ÿ   ß   Ã ã Ẽ ẽ Ĩ ĩ Ñ ñ Õ õ Ũ ũ Ỹ ỹ   Ç ç Ģ ģ Ķ ķ Ļ ļ Ņ ņ Ŗ ŗ Ş ş Ţ ţ   Đ đ   Ů ů   Ǎ ǎ Č č Ď ď Ě ě Ǐ ǐ Ľ ľ Ň ň Ǒ ǒ Ř ř Š š Ť ť Ǔ ǔ Ž ž   Ā ā Ē ē Ī ī Ō ō Ū ū Ȳ ȳ Ǣ ǣ   ǖ ǘ ǚ ǜ Ă ă Ĕ ĕ Ğ ğ Ĭ ĭ Ŏ ŏ Ŭ ŭ   Ċ ċ Ė ė Ġ ġ İ ı Ż ż   Ą ą Ę ę Į į Ǫ ǫ Ų ų   Ḍ ḍ Ḥ ḥ Ḷ ḷ Ḹ ḹ Ṃ ṃ Ṇ ṇ Ṛ ṛ Ṝ ṝ Ṣ ṣ Ṭ ṭ   Ł ł   Ő ő Ű ű   Ŀ ŀ   Ħ ħ   Ð ð Þ þ   Œ œ   Æ æ Ø ø Å å   Ə ə      Greek: Ά ά Έ έ Ή ή Ί ί Ό ό Ύ ύ Ώ ώ   Α α Β β Γ γ Δ δ   Ε ε Ζ ζ Η η Θ θ   Ι ι Κ κ Λ λ Μ μ   Ν ν Ξ ξ Ο ο Π π   Ρ ρ Σ σ ς Τ τ Υ υ   Φ φ Χ χ Ψ ψ Ω ω      Cyrillic: А а Б б В в Г г   Ґ ґ Ѓ ѓ Д д Ђ ђ   Е е Ё ё Є є Ж ж   З з Ѕ ѕ И и І і   Ї ї Й й Ј ј К к   Ќ ќ Л л Љ љ М м   Н н Њ њ О о П п   Р р С с Т 

In [105]:
print(bag_o_words)

[['Ahh', 'Hello', 'Witzeman', 'Symbols', 'm²', 'm³', 'Characters', 'Greek', 'Cyrillic', 'IPA', 'My', 'Famous', 'Article', 'witze', 'happiness', 'wikipedia', 'The', 'Witzeman', 'is', 'great', 'honour', 'that', 'has', 'been', 'passed', 'down', 'through', 'the', 'generations', 'of', 'many', 'families', 'regardless', 'of', 'race', 'age', 'character', 'or', 'knowledge', 'The', 'outside', 'world', 'knows', 'little', 'about', 'these', 'elusive', 'characters', 'and', 'the', 'honour', 'of', 'the', 'Witzeman', 'For', 'those', 'who', 'have', 'ever', 'been', 'Witzeman', 'it', 'has', 'been', 'said', 'to', 'have', 'been', 'great', 'honour', 'although', 'the', 'qualities', 'needed', 'for', 'the', 'job', 'have', 'never', 'been', 'disclosed', 'person', 'may', 'not', 'now', 'they', 'were', 'Witzeman', 'for', 'many', 'years', 'until', 'they', 'are', 'called', 'by', 'former', 'Witzemans', 'via', 'dreams', 'Associates', 'of', 'the', 'Witzeman', 'are', 'known', 'to', 'be', 'certain', 'Babe', 'Cool', 'or', '